### This notebook scrapes all listings.csv.gz united-states data from the insideairbnb get data page and combines them.

In [1]:
import pandas as pd
import re
import os
import urllib
import numpy as np

Read html page with links to each indivdual csv

In [2]:
with open("../00_data/Get the Data - Inside Airbnb. Adding data to the debate..html", encoding='utf8') as file:
    lines = file.readlines()

Get relevant links

In [4]:
relevantlines =  [line for line in lines if bool(re.search(r"listings\.csv\.gz\"", line))] #subset lines with listings.csv but not gz
links = [re.findall(r"href=\".*listings\.csv\.gz\"", line)[0][6:-1] for line in relevantlines] #extract the links
links = [l for l in links if bool(re.search(r"united-states", l)) ] #subset for united-states

Download files to environment

In [31]:
files = []
for i,link in enumerate(links):
    try:
        files.append(pd.read_csv(link))
        del files[-1]['license']
        pass
    except:
        files.append(None)

C:\Users\satvi\Anaconda3\envs\ids701\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Clean out links that did not work

In [32]:
bad_links = [i for i,f in enumerate(files) if f is None]
links = [link for i,link in enumerate(links) if i not in bad_links]
files = [file for i,file in enumerate(files) if i not in bad_links]

Extract file information and add to files

In [33]:
regions = [link.split("/")[-4] for link in links ]
dates = [link.split("/")[-3] for link in links ]
dates = [int(date.split("-")[1]) for date in dates]
for i,file in enumerate(files):
    file['region'] = regions[i]
    file['month'] = dates[i]

#### Verify columns

In [34]:
unique_columns = set() #All unique column names
unique_n_columns = set() #All unique number of columns per dataset
for file in files:
    unique_n_columns.add(len(file.columns.values))
    for col in file.columns.values:
        unique_columns.add(col)
        pass
    pass
print("Unique Columns: ", unique_columns)
print("Unique number of Columns: ", unique_n_columns)

Unique Columns:  {'review_scores_checkin', 'availability_365', 'neighbourhood', 'availability_60', 'number_of_reviews', 'number_of_reviews_ltm', 'review_scores_value', 'host_id', 'review_scores_location', 'property_type', 'longitude', 'host_response_time', 'calculated_host_listings_count', 'maximum_nights', 'beds', 'host_is_superhost', 'review_scores_communication', 'host_about', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_shared_rooms', 'amenities', 'price', 'name', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'review_scores_cleanliness', 'host_identity_verified', 'host_since', 'number_of_reviews_l30d', 'listing_url', 'host_listings_count', 'neighbourhood_cleansed', 'maximum_minimum_nights', 'host_total_listings_count', 'host_verifications', 'calendar_updated', 'review_scores_accuracy', 'last_scraped', 'scrape_id', 'availability_90', 'host_location', 'host_thumbnail_url', 'last_review', 'neighborhood_overview', 'host_neighbourhood', 'has_avail

Investigate missing columns

In [36]:
missing_cols = [[col for col in unique_columns if col not in f.columns.values] for f in files]
missing_cols = [mc for mc in missing_cols if len(mc) != 0]
missing_cols_set = set()
for mcs in missing_cols:
    for mc in mcs:
        missing_cols_set.add(mc)
print("Columns that can go missing are: ", missing_cols_set)

Columns that can go missing are:  set()


Thus, we observe all indivual files to have the same 76 columns 

Concatenate and export

In [37]:
final_csv = pd.concat(files, ignore_index=True)

In [38]:
final_csv.to_csv("../00_data/combined.csv")